In [1]:
import pandas as pd

# 8.1 Parsing Unix timestamps

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

Here's an [explanation of how this file works](http://popcon.ubuntu.com/README).

_NB: for first-time use or to reconfigure, use_ `sudo dpkg-reconfigure popularity-contest`


### Poking around with this data set 
If not working with Linux with popularity-contest then use the stock file in `../data/popularity-contest` and uncomment the below 

In [ ]:
# popcon = pd.read_csv('../data/popularity-contest', sep=' ', )
# popcon

Using part of the answer of [Keith](https://stackoverflow.com/questions/48946492/capturing-terminal-output-into-pandas-dataframe-without-creating-external-text-f/48947510#48947510) and the one found [here](https://stackoverflow.com/questions/22604564/how-to-create-a-pandas-dataframe-from-a-string), to pass information from string to pandas dataframe.

In [60]:
import sys
import subprocess
from io import StringIO

cmd = ['popularity-contest']
a = subprocess.Popen(cmd, stdout=subprocess.PIPE)
b = StringIO(a.communicate()[0].decode('utf-8'))
popcon = pd.read_csv(b, sep=' ')
popcon

,POPULARITY-CONTEST-0,TIME:1611549535,ID:0819bd4d3a804c7e9f6b6f9e463bcd46,ARCH:amd64,POPCONVER:1.69ubuntu1,VENDOR:Ubuntu
0,1611532800,1610884800,libssl1.1,/usr/lib/x86_64-linux-gnu/libcrypto.so.1.1,NaN,NaN
1,1611532800,1610884800,perl-base,/usr/bin/perl,NaN,NaN
2,1611532800,1610884800,openssh-client,/usr/bin/ssh-agent,NaN,NaN
3,1611532800,1610884800,libc-bin,/usr/lib/locale/C.UTF-8/LC_TIME,NaN,NaN
4,1611532800,1610884800,libc6,/usr/lib/x86_64-linux-gnu/gconv/gconv-modules....,NaN,NaN
...,...,...,...,...,...,...
718,0,0,libgusb2,<NOFILES>,NaN,NaN
719,0,0,libpopt0,<NOFILES>,NaN,NaN
720,0,0,libcryptsetup12,<NOFILES>,NaN,NaN
721,0,0,xkb-data,<NOFILES>,NaN,NaN


The last column (VENDOR:Ubuntu) is always NaN and not useful here, so remove it with `popcon.drop('VENDOR:Ubuntu', axis=1, inplace=True)` (or with `del popcon['VENDOR:Ubuntu']`).

The data set has both a header (which read_csv used) and a trailer which marks the end of the dataset. We can remove this with the slicer [:-1]. Check its shape and tail

In [61]:
# popcon.drop('VENDOR:Ubuntu', axis=1, inplace=True)
popcon.drop(columns=['VENDOR:Ubuntu'], inplace=True)
popcon = popcon[:-1]
popcon.shape

(722, 5)

In [62]:
# Trimmed tail
popcon.tail()

,POPULARITY-CONTEST-0,TIME:1611549535,ID:0819bd4d3a804c7e9f6b6f9e463bcd46,ARCH:amd64,POPCONVER:1.69ubuntu1
717,0,0,libvisual-0.4-0,<NOFILES>,NaN
718,0,0,libgusb2,<NOFILES>,NaN
719,0,0,libpopt0,<NOFILES>,NaN
720,0,0,libcryptsetup12,<NOFILES>,NaN
721,0,0,xkb-data,<NOFILES>,NaN


### Put it together
Read it, remove the last row, and rename the columns

In [63]:
# Read it, and remove the last row
#popcon = pd.read_csv('../data/popularity-contest', sep=' ', )[:-1]

import sys
import subprocess
from io import StringIO

cmd = ['popularity-contest']
a = subprocess.Popen(cmd, stdout=subprocess.PIPE)
b = StringIO(a.communicate()[0].decode('utf-8'))
popcon = pd.read_csv(b, sep=' ')
popcon = popcon[:-1]
del popcon['VENDOR:Ubuntu']

# rename the columns
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']
popcon.shape

(722, 5)

The colums are the access time, created time, package name, recently used program, and a tag

In [65]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1611532800,1610884800,perl-base,/usr/bin/perl,NaN
1,1611532800,1610884800,libc-bin,/usr/lib/locale/C.UTF-8/LC_TIME,NaN
2,1611532800,1610884800,libssl1.1,/usr/lib/x86_64-linux-gnu/libcrypto.so.1.1,NaN
3,1611532800,1610884800,libc6,/usr/lib/x86_64-linux-gnu/gconv/gconv-modules....,NaN
4,1611532800,1610884800,openssh-client,/usr/bin/ssh-agent,NaN


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [66]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [67]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [68]:
# popcon['atime'].dtype
popcon[['atime', 'ctime']].dtypes

atime    datetime64[ns]
ctime    datetime64[ns]
dtype: object

So now we can look at our `atime` and `ctime` as dates!

In [69]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,2021-01-25,2021-01-17 12:00:00,perl-base,/usr/bin/perl,NaN
1,2021-01-25,2021-01-17 12:00:00,libc-bin,/usr/lib/locale/C.UTF-8/LC_TIME,NaN
2,2021-01-25,2021-01-17 12:00:00,libssl1.1,/usr/lib/x86_64-linux-gnu/libcrypto.so.1.1,NaN
3,2021-01-25,2021-01-17 12:00:00,libc6,/usr/lib/x86_64-linux-gnu/gconv/gconv-modules....,NaN
4,2021-01-25,2021-01-17 12:00:00,openssh-client,/usr/bin/ssh-agent,NaN


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.

In [70]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [71]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [72]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
15,2021-01-23,2021-01-23,update-manager-core,/usr/bin/ubuntu-security-status,<RECENT-CTIME>
9,2021-01-23,2021-01-23,ubuntu-drivers-common,/usr/lib/ubiquity/target-config/31ubuntu_drive...,<RECENT-CTIME>
14,2021-01-23,2021-01-23,ubuntu-release-upgrader-core,/usr/bin/do-release-upgrade,<RECENT-CTIME>
13,2021-01-23,2021-01-23,update-notifier-common,/usr/lib/update-notifier/apt-check,<RECENT-CTIME>
12,2021-01-23,2021-01-23,alsa-utils,/usr/bin/arecord,<RECENT-CTIME>
10,2021-01-23,2021-01-23,apt,/usr/lib/apt/planners/dump,<RECENT-CTIME>
11,2021-01-23,2021-01-23,python3-xkit,/usr/lib/python3/dist-packages/xkit-0.0.0.egg-...,<RECENT-CTIME>
8,2021-01-23,2021-01-23,apt-utils,/usr/lib/apt/solvers/apt,<RECENT-CTIME>
7,2021-01-23,2021-01-23,cloud-init,/usr/lib/python3/dist-packages/cloudinit/warni...,<RECENT-CTIME>
6,2021-01-23,2021-01-23,python3-distupgrade,/usr/lib/python3/dist-packages/DistUpgrade/xor...,<RECENT-CTIME>


#original story: Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

It shows what I installed recently

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  